In [282]:
import json
import pandas as pd
import numpy as np
import csv

In [283]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reading some lines

In [284]:
c=0
max=10
with open("/content/drive/MyDrive/Titulacion/Ecuador data set exploration/ecuador_tbcov.tsv",'r') as tsv_file:
  tsv_reader=csv.reader(tsv_file, delimiter='\t')
  for line in tsv_reader:
    #  if line[0] == 'tweet_id':
    #     print(line)
    #  if line[0] == '1223956054226210817':
    #      print(line)
    print(line)
    c+=1
    if c==max+1:
      break

['tweet_id', 'date_time', 'lang', 'user_id', 'retweeted_id', 'quoted_id', 'in_reply_to_id', 'sentiment_conf', 'sentiment_label', 'user_type', 'gender_label', 'tweet_text_named_entities', 'geo_coordinates_lat_lon', 'geo_country_code', 'geo_state', 'geo_county', 'geo_city', 'place_bounding_box', 'place_country_code', 'place_state', 'place_county', 'place_city', 'user_loc_toponyms', 'user_loc_country_code', 'user_loc_state', 'user_loc_county', 'user_loc_city', 'user_profile_description_toponyms', 'user_profile_description_country_code', 'user_profile_description_state', 'user_profile_description_county', 'user_profile_description_city', 'tweet_text_toponyms', 'tweet_text_country_code', 'tweet_text_state', 'tweet_text_county', 'tweet_text_city']
['1223395538155995136', '2020-02-01T00:00:00', 'en', '296005306', '', '', '', '0.814', '1', 'ORG', '', "[{'label': 'ORG', 'term': 'UnitedStates Commerce'}, {'label': 'PERSON', 'term': 'WilburRoss'}, {'label': 'GPE', 'term': 'China'}]", '', '', '', 

c is the limit of of lines read

In [285]:
c = 0
max = 10000000 #10000000  7ceros
#nodes = set() 
Nodes = {}
with open("/content/drive/MyDrive/Titulacion/Ecuador data set exploration/ecuador_tbcov.tsv",'r') as tsv_file:
  tsv_reader=csv.reader(tsv_file, delimiter='\t')
  edgelist = []
  for line in tsv_reader:
    
    #Taking only Quotes
    if line[5] != '' and c > 1 and line[4] == '': #line[5] -> quoted_id   line[4] -> retweeted_id
      ## For save links
      time = line[1] #line[1] -> date_time
      source   = line[5]
      target   = line[0]  
      type_tw   = 'quoted_id'
      #time_str = time.isoformat(timespec='seconds')
      edgelist.append({"source":source,"target":target,"type_tw":type_tw, "time":time})

      ##For save nodes
      if source not in Nodes.keys(): # I will have reapeted sources, I use this to create once
        Nodes[source] = [] #if is a root is created without parameters
      if target in Nodes.keys(): # If a target exist maybe was a sorce before so Now I know the type and time
        Nodes[source].append(time)
      Nodes[target] = [time]#[type_tw,time]
    
    #Taking only replies
    if line[6] != '' and c > 1 and line[4] == '': #line[6] -> in_reply_to_id   line[4] -> retweeted_id
      ## For save links
      time = line[1] # line[1] -> date_time 
      source   = line[6]
      target   = line[0]  
      type_tw   = 'in_reply_to_id'
      edgelist.append({"source":source,"target":target,"type_tw":type_tw, "time":time})

      ## For save nodes
      if source not in Nodes.keys(): # I will have reapeted sources, I use this to create once
        Nodes[source] = []
      if target in Nodes.keys(): # If a target exist maybe was a sorce before so Now I know the type and time
        Nodes[source].append(time)
      Nodes[target] = [time]#[type_tw,time]

    c+=1
    if c == max:
      break

In [286]:
len(Nodes)

1044542

In [287]:
for i,j in Nodes.items():
    if i == '1223691644563836929':
        print(i,j)
    if i == '1223956054226210817':
        print(i,j)

1223691644563836929 []
1223956054226210817 ['2020-02-02T13:07:17']


OJO LSA SIGUIENTE LINEA PARA MAS LUEGO PORQUE QUIERO PONERLE UNA FECHA APROXIMADA A LA DEL HIJO MAS JOVEN

In [288]:
# for i,j in Nodes.items():#there are nodes with no date and tweet info
#   if len(j) == 0:
#     #Nodes[i].append('origin')
#     Nodes[i].append('2020-01-31T23:59:00')

To see if I have reapet nodes - different type

In [289]:
 #Nodes['1223415067137298432']

OJO LO SIGUIENTE  PARA MAS LUEGO PORQUE QUIERO PONERLE UNA FECHA APROXIMADA A LA DEL HIJO MAS JOVEN

In [290]:
# list_nodes=[]

In [291]:
# for i in Nodes.keys():
#   list_nodes.append({"id":i, "time":Nodes[i][0]})#"time":NodesR[i][1] , "type_tw":NodesR[i][0]})

In [292]:
# len(list_nodes)

In [293]:
# list_nodes[:5]

Edge List

In [294]:
len(edgelist)

656680

In [295]:
edgelist[:2]

[{'source': '1223369797875924993',
  'target': '1223399190606184448',
  'type_tw': 'in_reply_to_id',
  'time': '2020-02-01T00:14:31'},
 {'source': '1223310437158133762',
  'target': '1223409634473660418',
  'type_tw': 'quoted_id',
  'time': '2020-02-01T00:56:01'}]

### Add Levels

In [296]:
class Node():
    def __init__(self, state, level):
        self.state = state
        self.parent = []
        self.child = []
        self.level = level
        self.ownconversation_id = []
        self.graphid = "0"
    def add_child(self, child):
        self.child.append(child)
    def change_level(self,level):
      self.level = level
    def change_parent(self,parent):
      self.parent.append(parent)
    def change_conv_id(self,conv_id):
      self.ownconversation_id.append(conv_id)
    def change_graph_id(self,graph_id):
      self.graphid = graph_id

In [297]:
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    # def contains_state(self, state):
    #     return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node


In [298]:
# creating the node objects
AllNodes={}
for i in edgelist:
  Source=i['source']
  Target=i['target']
  if Source not in AllNodes.keys() and Target not in AllNodes.keys(): #facil cuando los dos no esten
    AllNodes[Source]=Node(state=Source, level=None)
    AllNodes[Source].add_child(Target)
    AllNodes[Target]=Node(state=Target, level=None)
    AllNodes[Target].change_parent(Source)
  elif Source in AllNodes.keys()  and Target not in AllNodes.keys(): # Si la fuente ya esta y el hijo no quiere decir que la fuente tiene otro hijo
    AllNodes[Source].add_child(Target)
    AllNodes[Target]=Node(state=Target, level=None)
    AllNodes[Target].change_parent(Source)
  elif Source not in AllNodes.keys() and Target  in AllNodes.keys(): #si la fuente no esta y el hijo si , quiere decir que ese hijo posiblemente antes fue source pero en realidad es un hijo de otro
    AllNodes[Source]=Node(state=Source, level=None)
    AllNodes[Source].add_child(Target)
    AllNodes[Target].change_parent(Source)
  elif  Source in AllNodes.keys() and Target  in AllNodes.keys(): #si los dos estan modifico sus padres e hijos
    AllNodes[Source].add_child(Target)
    AllNodes[Target].change_parent(Source)

In [299]:
AllNodes["1224051847931072513"].child

['1224054092835835909']

In [300]:
for i in AllNodes:
  if AllNodes[i].parent == []: #None
    frontier = QueueFrontier()
    frontier.add(AllNodes[i])
    # Keep track of number of levels explored
    num_explored = 0
    AllNodes[i].change_level(num_explored) 
  
    ## TO save de conv id in the origin
    conv_id = AllNodes[i].state
    AllNodes[i].change_conv_id(conv_id)
    while True:
      if frontier.empty():
            break
      # Choose a node from the frontier
      node = frontier.remove()
      #num_explored += 1

      for j in node.child:
        frontier.add(AllNodes[j])

        AllNodes[j].change_conv_id(conv_id)#to save the conv id in each child

        ## some tweets are from to origins so i will save the highest level
        if AllNodes[j].level != None:
          if AllNodes[j].level <  node.level+1: #para que se guarde el nivel mas grande
            AllNodes[j].change_level(node.level+1)

        elif AllNodes[j].level == None: 
          AllNodes[j].change_level(node.level+1)

In [301]:
# ESTO NO SIRVE
# c=0
# for i in AllNodes:
#   if AllNodes[i].level == 2:
#     print(AllNodes[i].state)
#     c=c+1
# print(c)

ESTO TAMBIEN MAS LUEGO ESTO AL FINAL DEBE SER PARA CREAR EL JSON

In [302]:
# list_nodes[:5]

In [303]:
# for i in range(len(list_nodes)):
#   list_nodes[i]['level']=AllNodes[list_nodes[i]['id']].level
#   #AllNodes[i['id']].level

In [304]:
# list_nodes[:5]

In [305]:
# for i in range(len(list_nodes)):
#   list_nodes[i]['conv_id']=AllNodes[list_nodes[i]['id']].ownconversation_id
#   #AllNodes[i['id']].level

In [306]:
# list_nodes[:5]

In [307]:
# ESTO NO SIRVE
# for i in range(len(list_nodes)):
#   if len(list_nodes[i]['conv_id'])>1 :
#     print(list_nodes[i]['conv_id'])


In [308]:
# for i in range(len(list_nodes)):
#   list_nodes[i]['childs']=AllNodes[list_nodes[i]['id']].child
#   #AllNodes[i['id']].level

In [309]:
# list_nodes[:5]

In [310]:
# len(list_nodes)

In [311]:
# for i in range(len(list_nodes)):
#   if list_nodes[i]['level']>1 :
#     print(list_nodes[i]['id'])
#     print(list_nodes[i]['level'])

Desde aqui con los enlaces (LO MISMO ESTO COMENTO PARA TENER AL FINAL)

In [312]:
# edgelist[:5]

In [313]:
# ESTO NO SIRVE
# for i in range(len(edgelist)): 
#   edgelist[i]['level']=AllNodes[edgelist[i]['target']].level
#   #AllNodes[i['id']].level

In [314]:
# for i in range(len(edgelist)):
#   edgelist[i]['conv_id']=AllNodes[edgelist[i]['target']].ownconversation_id
#   #AllNodes[i['id']].level

In [315]:
# edgelist[:5]

In [316]:
# ESTO NO SIRVE
# for i in range(len(edgelist)):
#   if len(edgelist[i]['conv_id'])>1 :
#     print(edgelist[i]['conv_id'])


### Compute Grafos 

In [317]:
# class Grafo(): # parece que no es necesario una clase para esto
#     def __init__(self, id):
#         self.id = id
#         self.nodes = set()

#     def add_node(self, node):
#         self.nodes.update(node)

In [318]:
grafos = {}

In [319]:
# grafos["0"] = Grafo("0") #aqui se crea totalmente uno nuevo

In [320]:
grafos["0"] = set() #aqui se crea totalmente uno nuevo

In [321]:
def get_branch(nodo):
  frontier = QueueFrontier()
  frontier.add(nodo)

  nodosrama =[]
  #print(nodo.state)
  nodosrama.append(nodo.state)
  # Keep track of number of levels explored
  num_explored = 0
    #AllNodes[i].change_level(num_explored) 

  ## TO save de conv id in the origin
  #conv_id = AllNodes[i].state
  #AllNodes[i].change_conv_id(conv_id)
  while True:
    if frontier.empty():
          break
    # Choose a node from the frontier
    node = frontier.remove()
    num_explored += 1

    for j in node.child:
      frontier.add(AllNodes[j])

      nodosrama.append(AllNodes[j].state)
  return nodosrama

In [ ]:
gindex=0
for i in AllNodes.keys():
  if AllNodes[i].parent == []:
    c=0

    for h in grafos.keys():
      #print("1era", AllNodes[i].state in grafos[h])
      #print("no esta", AllNodes[i].state not in grafos[h])
      # if AllNodes[i].state in grafos[h]:
      #   #grafos[h].add_node(AllNodes[i].state)
      #   #grafos[h].update(AllNodes[i].parent)
      #   grafos[h].update(AllNodes[i].child)
      #   break
      # elif (AllNodes[i].state not in grafos[h]):
        # vparent=0
        # for j in AllNodes[i].parent:
        #   if (j in grafos[h]):
        #     grafos[h].update([AllNodes[i].state])
        #     grafos[h].update(AllNodes[i].parent)
        #     grafos[h].update(AllNodes[i].child)
        #     vparent=1
        #     break
        # if vparent == 1:
        #   break
      vchild = 0 
      listbranch = get_branch(AllNodes[i])
      # if AllNodes[i].state == '1225267360849973248' and h=='1667':
      #   print("este nodo: ",1225267360849973248)
      #   print(listbranch)
      #   print('grafos[h]', grafos[h])

      #print("listbranch", listbranch)
      for j in listbranch:
        #print("j in grafos[h]: ",j in grafos[h])
        #print("grafos[h]", grafos[h])
        if (j in grafos[h]):
          grafos[h].update(listbranch)
          
          #print("listbranch", listbranch)
          #grafos[h].update(AllNodes[i].parent)
          #grafos[h].update(AllNodes[i].child)
          vchild = 1
          break
      if vchild == 1:
        break
      c=c+1
      #print(c == len(grafos.keys()))
    if c == len(grafos.keys()):
      gindex += 1
      grafos[str(gindex)] = set()
      listbranch = get_branch(AllNodes[i])
      grafos[str(gindex)].update(listbranch)

In [ ]:
# gindex=0 #NOOOO
# for i in AllNodes.keys():
#   c=0
#   for h in grafos.keys():
#     #print(AllNodes[i].state not in grafos[h].nodes)
#     if AllNodes[i].state in grafos[h]:
#       #grafos[h].add_node(AllNodes[i].state)
#       grafos[h].update(AllNodes[i].parent)
#       grafos[h].update(AllNodes[i].child)
#       break
#     elif (AllNodes[i].state not in grafos[h]):
#       vparent=0
#       for j in AllNodes[i].parent:
#          if (j in grafos[h]):
#            grafos[h].update([AllNodes[i].state])
#            grafos[h].update(AllNodes[i].parent)
#            grafos[h].update(AllNodes[i].child)
#            vparent=1
#            break
#       if vparent == 1:
#         break
#       vchild = 0 
#       for j in AllNodes[i].child:
#         if (j in grafos[h]):
#            grafos[h].update([AllNodes[i].state])
#            grafos[h].update(AllNodes[i].parent)
#            grafos[h].update(AllNodes[i].child)
#            vchild = 1
#            break
#       if vchild == 1:
#         break
#     c=c+1
#     #print(c == len(grafos.keys()))
#   if c == len(grafos.keys()):
#     gindex += 1
#     grafos[str(gindex)] = set()
#     grafos[str(gindex)].update([i])

In [ ]:
# grafos['1667']

In [ ]:
# grafos['1861']

In [ ]:
# get_branch(AllNodes['1225267360849973248'])

In [ ]:
# get_branch(AllNodes['1226163900120522752'])

In [ ]:
1226281212609691649 == 1226281212609691649

In [ ]:
# print(AllNodes['1226281212609691649'].state) ##este es el problema
# print(AllNodes['1226281212609691649'].parent)
# print(AllNodes['1226281212609691649'].child)

In [ ]:
# print(AllNodes['1226163900120522752'].state)
# print(AllNodes['1226163900120522752'].parent)
# print(AllNodes['1226163900120522752'].child)

In [ ]:
# grafos['1667']

In [ ]:
len(grafos)

### Unir Grafos que tienen un nodo en comun

In [ ]:
g = { '1' : [1,2,3], '2': [4,5,6], '3': [1,7,8], '4': [8,9]}

In [ ]:
def merge_lists(d):
    src = dict((k, set(v)) for k, v in d.items())

    while True:
        dest = {}
        count = 0
        while src:
            k1, temp = src.popitem()
            if temp is None:
                continue
            for k2, v in src.items():
                if v is None:
                    continue
                if temp & v:
                    temp |= v
                    src[k2] = None
                    count += 1
                    k1 = min(k1, k2)
            dest[k1] = temp

        if count > 0:
            #print count
            #print_dict(dest)
            src = dest
        else:
            dest = dict((k, sorted(list(v))) for k, v in dest.items())
            return dest

In [ ]:
print(merge_lists(g))

In [ ]:
AllGraphs = merge_lists(grafos)

In [ ]:
print(len(grafos))
print(len(AllGraphs))

In [ ]:
#AllGraphs['1861']

### Add Grafos ids a los nodos

In [ ]:
# for i in AllNodes:
#   if AllNodes[i].level == 2:
#     print("hola")

In [ ]:
for i in AllGraphs:
  for j in AllGraphs[i]:
    AllNodes[j].change_graph_id(i)

### Try this to delete the graphs that have only paths of one

In [ ]:
No=[]
for i in AllGraphs:
  bandera = False
  for j in AllGraphs[i]:
    if AllNodes[j].level > 1:
      bandera=True
  if bandera == False:
    No.append(i)

In [ ]:
'173' in No

In [ ]:
len(No)

### Prepare lists for the json file

In [ ]:
for i,j in Nodes.items():#there are nodes with no date and tweet info
  if len(j) == 0:
    #Nodes[i].append('origin')
    Nodes[i].append('2020-01-31T23:59:00')

In [ ]:
 list_nodes=[]

In [ ]:
for i in Nodes.keys():
  list_nodes.append({"id":i, "time":Nodes[i][0]})#"time":NodesR[i][1] , "type_tw":NodesR[i][0]})

In [ ]:
len(list_nodes)

In [ ]:
list_nodes[:5]

In [ ]:
for i in range(len(list_nodes)):
  list_nodes[i]['level']=AllNodes[list_nodes[i]['id']].level
  #AllNodes[i['id']].level

In [ ]:
for i in range(len(list_nodes)):
  list_nodes[i]['conv_id']=AllNodes[list_nodes[i]['id']].ownconversation_id
  #AllNodes[i['id']].level

In [ ]:
for i in range(len(list_nodes)):
  list_nodes[i]['childs']=AllNodes[list_nodes[i]['id']].child
  #AllNodes[i['id']].level

In [ ]:
for i in range(len(list_nodes)):
  list_nodes[i]['graphid']=AllNodes[list_nodes[i]['id']].graphid
  #AllNodes[i['id']].level

In [ ]:
len(list_nodes)

In [ ]:
list_nodes[:5]

Desde aqui los EDGES

In [ ]:
edgelist[:5]

In [ ]:
for i in range(len(edgelist)):
  edgelist[i]['conv_id']=AllNodes[edgelist[i]['target']].ownconversation_id
  #AllNodes[i['id']].level

In [ ]:
for i in range(len(edgelist)):
  edgelist[i]['graphid']=AllNodes[edgelist[i]['target']].graphid
  #AllNodes[i['id']].level

In [ ]:
len(edgelist)

In [ ]:
edgelist[:5]

### Save the json for d3

In [ ]:
data ={"nodes": list_nodes ,"links":edgelist}

In [ ]:
#data

In [ ]:
len(data["nodes"])

In [ ]:
len(data["links"])

 10 000 entries -> 1324 nodes \ 821 edges \\
 100 000 entries ->  12570 nodes \  7961 edges \\
 1 000 000 entries ->  137772 nodes \  87614 edges

In [ ]:
json_string = json.dumps(data)

In [ ]:
with open('/content/drive/MyDrive/Titulacion/Ecuador data set exploration/10000000entriesNW23AG2022.json', 'w') as outfile:
    outfile.write(json_string)

Filtered

In [ ]:
list_nodesft = []
edgelistft = []

for i in list_nodes:
  if i['graphid'] not in No:
    list_nodesft.append(i)

for i in edgelist:
  if i['graphid'] not in No:
    edgelistft.append(i)

In [ ]:
len(list_nodesft)

In [ ]:
len(edgelistft)

 10 000 entries -> 58 nodes \ 47 edges \\
  1 000 000 entries -> 9643 nodes \ 47 edgelistft \\

In [ ]:
dataft ={"nodes": list_nodesft ,"links":edgelistft}

In [ ]:
json_stringft = json.dumps(dataft)

In [ ]:
with open('/content/drive/MyDrive/Titulacion/Ecuador data set exploration/10000000entriesNW23AG2022ft.json', 'w') as outfile:
    outfile.write(json_stringft)

### Ahora para quitar todos los nodos que pertenezcan a un grafo con maximo camino de 1

In [ ]:
Over1Convs_ids = []

In [ ]:
for i in AllNodes:
  if AllNodes[i].parent == []: #None
    frontier = QueueFrontier()
    frontier.add(AllNodes[i])
    # Keep track of number of levels explored
    num_explored = 0
    #AllNodes[i].change_level(num_explored) 
  
    ## TO save de conv id in the origin
    #conv_id = AllNodes[i].state
    #AllNodes[i].change_conv_id(conv_id)
    while True:
      if frontier.empty():
            break
      # Choose a node from the frontier
      node = frontier.remove()
      #num_explored += 1

      for j in node.child:
        frontier.add(AllNodes[j])

        #AllNodes[j].change_conv_id(conv_id)#to save the conv id in each child

        ## some tweets are from to origins so i will save the highest level
        if AllNodes[j].level > 1:
            #print(AllNodes[j].level)
            #print(AllNodes[j].ownconversation_id)
            for cid in AllNodes[j].ownconversation_id:
                if cid not in Over1Convs_ids:
                    Over1Convs_ids.append(cid)

In [ ]:
# Over1Convs_ids

In [ ]:
NewNodes = {}
for i in AllNodes:
     for j in AllNodes[i].ownconversation_id:
            if j in Over1Convs_ids:
                NewNodes[i] = AllNodes[i]
                break

In [ ]:
## Ahora la Lista de nodos

In [ ]:
Newlist_nodes = []

In [ ]:
for i in range(len(list_nodes)):
    if ( (list_nodes[i]["id"] in NewNodes.keys()) ):
        Newlist_nodes.append(list_nodes[i])

In [ ]:
Newlist_nodes[:5]

[{'childs': ['1223693137849266176',
   '1223934528986677248',
   '1223956054226210817',
   '1223960200492789761',
   '1223965803416051713',
   '1223968949408292864',
   '1223971997681639424',
   '1223976950609457153',
   '1224000780409221123',
   '1224010648423739394',
   '1224065598772195328',
   '1224087702909472775',
   '1224105763792138242',
   '1224297684976242689',
   '1224485245640151040'],
  'conv_id': ['1223691644563836929'],
  'id': '1223691644563836929',
  'level': 0,
  'time': '2020-01-31T23:59:00'},
 {'childs': [],
  'conv_id': ['1223691644563836929'],
  'id': '1223693137849266176',
  'level': 1,
  'time': '2020-02-01T19:42:33'},
 {'childs': [],
  'conv_id': ['1223691644563836929'],
  'id': '1223934528986677248',
  'level': 1,
  'time': '2020-02-02T11:41:45'},
 {'childs': ['1223990893927194624', '1224001598323675136'],
  'conv_id': ['1223691644563836929'],
  'id': '1223956054226210817',
  'level': 1,
  'time': '2020-02-02T13:07:17'},
 {'childs': [],
  'conv_id': ['12236916

In [ ]:
len(Newlist_nodes)

1448

In [ ]:
##### Ahora para los edges

In [ ]:
Newedgelist = []

In [ ]:
edgelist[:5]

[{'conv_id': ['1223369797875924993'],
  'source': '1223369797875924993',
  'target': '1223399190606184448',
  'time': '2020-02-01T00:14:31',
  'type_tw': 'in_reply_to_id'},
 {'conv_id': ['1223310437158133762'],
  'source': '1223310437158133762',
  'target': '1223409634473660418',
  'time': '2020-02-01T00:56:01',
  'type_tw': 'quoted_id'},
 {'conv_id': ['1223288485311705088', '1223364820084252672'],
  'source': '1223288485311705088',
  'target': '1223415067137298432',
  'time': '2020-02-01T01:17:36',
  'type_tw': 'quoted_id'},
 {'conv_id': ['1223288485311705088', '1223364820084252672'],
  'source': '1223364820084252672',
  'target': '1223415067137298432',
  'time': '2020-02-01T01:17:36',
  'type_tw': 'in_reply_to_id'},
 {'conv_id': ['1223358934276300800'],
  'source': '1223358934276300800',
  'target': '1223416791768223744',
  'time': '2020-02-01T01:24:27',
  'type_tw': 'quoted_id'}]

In [ ]:
for i in range(len(edgelist)):
    if (edgelist[i]["source"] in NewNodes.keys()) or (edgelist[i]["target"] in NewNodes.keys()):
        Newedgelist.append(edgelist[i])
    #edgelist[i]['conv_id']=AllNodes[edgelist[i]['target']].ownconversation_id
  #AllNodes[i['id']].level

In [ ]:
Newedgelist[:2]

[{'conv_id': ['1223691644563836929'],
  'source': '1223691644563836929',
  'target': '1223693137849266176',
  'time': '2020-02-01T19:42:33',
  'type_tw': 'quoted_id'},
 {'conv_id': ['1223691644563836929'],
  'source': '1223691644563836929',
  'target': '1223934528986677248',
  'time': '2020-02-02T11:41:45',
  'type_tw': 'quoted_id'}]

In [ ]:
len(Newedgelist)

1352

In [ ]:
for i in Newedgelist:
  if i['source'] not in NewNodes.keys():
    print('source', i)
  if i['target'] not in NewNodes.keys():
    print('target', i)

source {'source': '1223989405884481537', 'target': '1224500429003853825', 'type_tw': 'quoted_id', 'time': '2020-02-04T01:10:27', 'conv_id': ['1223989405884481537', '1224326972328890368']}
source {'source': '1224870656833216514', 'target': '1224881556046733312', 'type_tw': 'quoted_id', 'time': '2020-02-05T02:24:54', 'conv_id': ['1224879039543959554', '1224870656833216514']}
source {'source': '1225465229305360387', 'target': '1226183495388712960', 'type_tw': 'in_reply_to_id', 'time': '2020-02-08T16:38:21', 'conv_id': ['1226168623854694400', '1225465229305360387']}
source {'source': '1225465228558774272', 'target': '1226183713161121792', 'type_tw': 'in_reply_to_id', 'time': '2020-02-08T16:39:13', 'conv_id': ['1226168623854694400', '1225465228558774272']}
source {'source': '1226192516363509760', 'target': '1226282482389131271', 'type_tw': 'in_reply_to_id', 'time': '2020-02-08T23:11:41', 'conv_id': ['1226163900120522752', '1226192516363509760']}
source {'source': '1226268960536121350', 'tar

In [ ]:
## saving

In [ ]:
data2 ={"nodes": Newlist_nodes ,"links":Newedgelist}

In [ ]:
json_string = json.dumps(data2)

In [ ]:
with open('/content/drive/MyDrive/Titulacion/Ecuador data set exploration/100000entriesNW07AG2022filtered.json', 'w') as outfile:
    outfile.write(json_string)